In [3]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from collections import Counter
from operator import itemgetter
import pandas as pd
from sqlalchemy import create_engine

# Calling the nltk package to merge the data of people without existing VIAF URI in the two datasets 
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder

from nltk.metrics import BigramAssocMeasures
from nltk.metrics import TrigramAssocMeasures
import numpy as np


In [123]:
query = """
PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?name ?sName ?uri ?year ?bio
WHERE
  {   { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1770" )
        FILTER ( ( ( ( ( regex(?bio, "juriste", "i") || regex(?bio, "professeur de droit", "i") ) || regex(?bio, "docteur en droit", "i") ) || regex(?bio, "avocat", "i") ) || regex(?bio, "juge", "i") ) || regex(?bio, "magistrat", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf.org", "i")
          }
      }
    UNION
      { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1770" )
        FILTER ( ( ( regex(?bio, "économiste") || regex(?bio, "Economiste") ) || regex(?bio, "professeur d'économie", "i") ) || regex(?bio, "docteur en économie", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri1
            FILTER regex(?uri1, "viaf.org", "i")
            BIND(strbefore(str(?uri1), "http://viaf.org/viaf/") AS ?uri)
          }
      }
  }
ORDER BY DESC(?uri)


"""

In [124]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql") ##, returnFormat=RDFXML)  [LOCALHOST]

In [125]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [126]:
try:
    rc_bnf = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [127]:
# Number of rows in the result
len(rc_bnf['results']['bindings'])

224

In [128]:
# Inspect the first three rows
i = 0
for l in rc_bnf['results']['bindings']:
    if i < 100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb16123308w#about'}, 'name': {'type': 'literal', 'value': 'Jacques-Simon Merlin'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/95365306'}, 'bd': {'type': 'literal', 'value': '1765-09-04'}, 'bio': {'type': 'literal', 'value': "Libraire. - Natif de Château-Porcien (Ardennes). Avocat de formation. Beau-frère du libraire parisien Jean-Baptiste-François Née de La Rochelle, dont il rachète la librairie en juin 1793 et à qui il succède. Intente un procès aux libraires Jean-Nicolas Barba et François Dufart en déc. 1798 puis aux libraires Jean-Charles Silvestre et Ponthieu en juin 1799, pour contrefaçon des œuvres de Jean-François Marmontel. Breveté libraire le 1er octobre 1812 (brevet renouvelé le 30 mars 1820). Spécialisé dans la librairie d'ancien. Rédacteur de nombreux catalogues de bibliothèques et de vente de livres. Encore en activité en juin 1834. Décédé en 1835 à Paris. Son fils et associé Romain Merlin (1793-1876)

In [10]:
result_bnf = []
for l in rc_bnf['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            sName = l['sName']['value']
        except Exception as e:
            sName = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            bio = l['bio']['value']
        except Exception as e:
            bio = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            year = ''
        result_bnf.append([l['s']['value'], uri, name, sName, year, bio])        
            
        

In [11]:
print(len(result_bnf))
result_bnf[:2]

11201


[['http://data.bnf.fr/ark:/12148/cb12981404c#about',
  'http://viaf.org/viaf/99996033',
  'Léon Garnier',
  '',
  '1836',
  "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"],
 ['http://data.bnf.fr/ark:/12148/cb13484444m#about',
  'http://viaf.org/viaf/9999219',
  'Gaston de Pawlowski',
  '',
  '1874',
  'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"']]

In [12]:
query_2= """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?uri ?name (year(xsd:dateTime(?Birth_Date)) AS ?year) ?abstract
WHERE
  {   { ?s  a              dbo:Economist ;
          # "Economist" has a class function, he explains the use of the "a".
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
          # We use this filter to preserve only the persons born after 1800. 
          # Here, the method is a little different because we have a date and not a year. 
          # We convert above the date to a year to have the same format as BnF Data.
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name }
        FILTER ( xsd:string(?name) != "Samuel Bowles" )
     # We use this filter to remove the data concerning "Samuel Bowles" 
     # because there is mistakes in data i.e he is related to wrong people -eg. William Turner (cf. https://dbpedia.org/page/Samuel_Bowles_(economist)).
     # We will can fix this problem later, by entering data about him manually.
      }
    UNION
      { ?s  ?propriety  dbr:Economist
      # Here, "Economist" is a instance, we have as triplet "subject predicate object". 
      # In DBpedia, "Economist" is define as a instance and at the same time as a class. So we use both.
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }    
      }
    UNION
      { ?s  ?p             dbr:Jurist ;
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
       # We use this filter to remove the data concerning "Cicero". 
       # He appears in results, certainly because his date of birth contains the sign "-" (cf. https://dbpedia.org/page/Cicero).
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
      }
    UNION
      { ?s  ?p             dbr:Lawyer ;
            dbp:birthDate  ?Birth_Date
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
      }
    UNION
      { ?s  a              dbr:Professor ;
            dbp:birthDate  ?Birth_Date ;
            dbo:abstract   ?abstract
        FILTER ( ( ( ( regex(?abstract, "lawyer", "i") || regex(?abstract, "jurist", "i") ) || regex(?abstract, "juriste", "i") ) || regex(?abstract, "attorney", "i") ) || regex(?abstract, "legal professional", "i") )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
      }
  }
ORDER BY DESC(?uri)
"""

In this query, we have made the choice to aggregate, by a UNION clause, several queries to maximise the results' number. Also we request the "economists" and the "jurists" in only one query. 

Obviously, we chose classes and instances directly related to our population, but also the "professor" instance, because some "economists" or "jurists" are in this instance (we have tried with and without them, and there more result when we use them). 

Also, we excluse all classes because they don't add more result, except the "Economist" class (we keep it) 

For exemple, we exclude the resource "personFunction" and the resource "Jurists" because they add no more data. Additionally, we keep only the "Professor" instance for the jurists (it returns result only for the jurists).

In [13]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [14]:
sparql.setQuery(query_2)
sparql.setReturnFormat(JSON)

In [15]:
rc_db = sparql.queryAndConvert()

In [16]:
# Number of rows in the result
len(rc_db['results']['bindings'])

8545

In [17]:
# Inspect the first three rows
i = 0
for l in rc_db['results']['bindings']:
    if i <100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/António_de_Almeida_Santos'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99921066'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'António de Almeida Santos'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1926'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Anita_Augspurg'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9976800'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Anita Augspurg'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1857'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Mason_Gaffney'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9960617'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Mason Gaffney'}, 'year': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1923'}}
{'s'

In [18]:
# Create a list with URI, VIAF URI, name, year
result_dbpedia = []
for l in rc_db['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            uri = ''
        result_dbpedia.append([l['s']['value'], uri, name, year])

In [19]:
# Inspect the first three of the list
result_dbpedia[:10]

[['http://dbpedia.org/resource/António_de_Almeida_Santos',
  'http://viaf.org/viaf/99921066',
  'António de Almeida Santos',
  '1926'],
 ['http://dbpedia.org/resource/Anita_Augspurg',
  'http://viaf.org/viaf/9976800',
  'Anita Augspurg',
  '1857'],
 ['http://dbpedia.org/resource/Mason_Gaffney',
  'http://viaf.org/viaf/9960617',
  'Mason Gaffney',
  '1923'],
 ['http://dbpedia.org/resource/Hermann_Heinrich_Gossen',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '1810'],
 ['http://dbpedia.org/resource/Gottfried_Haberler',
  'http://viaf.org/viaf/99257315',
  'Gottfried Haberler',
  '1900'],
 ['http://dbpedia.org/resource/Michael_C._Burda',
  'http://viaf.org/viaf/9922987',
  'Michael C. Burda',
  '1959'],
 ['http://dbpedia.org/resource/Xavier_Vives',
  'http://viaf.org/viaf/9920331',
  'Xavier Vives',
  '1955'],
 ['http://dbpedia.org/resource/Vittorio_Emanuele_Orlando',
  'http://viaf.org/viaf/9914155',
  'Vittorio Emanuele Orlando',
  '1860'],
 ['http://dbpedia.org/reso

In [20]:
query_3= """
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  wd:   <http://www.wikidata.org/entity/>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT  ?s ?uri ?name ?year
WHERE
  {   { ?s  wdt:P106  wd:Q188094 } # Economists
    OPTIONAL
      { ?s  wdt:P569  ?dob } # date of birth
    OPTIONAL
      { ?s  wdt:P214  ?oldURI
        BIND(uri(concat("http://viaf.org/viaf/", strafter(str(?oldURI), ""))) AS ?uri) # It's useful to have the URI VIAF in the same and merge data.
      }
    BIND(year(?dob) AS ?year)
    FILTER ( ?year > 1770 )
    OPTIONAL
      { ?s  rdfs:label  ?name
        FILTER ( lang(?name) = "en" )
      }
  }
"""

In [21]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [22]:
sparql.setQuery(query_3)
sparql.setReturnFormat(JSON)

In [23]:
rc_wk = sparql.queryAndConvert()

In [24]:
# Number of rows in the result
len(rc_wk['results']['bindings'])

29412

In [25]:
# Create a list with URI, VIAF URI, name, year
result_wikidata = []
for l in rc_wk['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            uri = ''
        result_wikidata.append([l['s']['value'], uri, name, year])

In [26]:
result_wikidata[:10]

[['http://www.wikidata.org/entity/Q65561',
  'http://viaf.org/viaf/232142151',
  'Hans Apel',
  '1932'],
 ['http://www.wikidata.org/entity/Q65589',
  'http://viaf.org/viaf/41917922',
  'Alexander Rüstow',
  '1885'],
 ['http://www.wikidata.org/entity/Q91640',
  'http://viaf.org/viaf/59144443',
  'Werner Hildenbrand',
  '1936'],
 ['http://www.wikidata.org/entity/Q91836',
  'http://viaf.org/viaf/30299539',
  'Max Hirsch',
  '1832'],
 ['http://www.wikidata.org/entity/Q91603',
  'http://viaf.org/viaf/40168611',
  'Joachim Friedrich',
  '1953'],
 ['http://www.wikidata.org/entity/Q66332',
  'http://viaf.org/viaf/67260519',
  'Georg Friedrich Knapp',
  '1842'],
 ['http://www.wikidata.org/entity/Q66838',
  'http://viaf.org/viaf/17372842',
  'Heinrich Freiherr von Stackelberg',
  '1905'],
 ['http://www.wikidata.org/entity/Q66624',
  'http://viaf.org/viaf/20037933',
  'Benedict Friedlaender',
  '1866'],
 ['http://www.wikidata.org/entity/Q66094',
  'http://viaf.org/viaf/47845158',
  'Jürgen Stark'

In [27]:
engine = create_engine('sqlite:///database.sqlite_2', echo=False)

In [28]:
df_bnf = pd.DataFrame([f for f in result_bnf if len(f[1]) > 0], columns=['uri_bnf', 'viaf', 'name_bnf', 'sName', 'year_bnf', 'bio_bnf'])
print(len(df_bnf))
df_bnf.fillna('')

df_bnf[:10]

9147


,uri_bnf,viaf,name_bnf,sName,year_bnf,bio_bnf
0,http://data.bnf.fr/ark:/12148/cb12981404c#about,http://viaf.org/viaf/99996033,Léon Garnier,,1836,Juriste. - Administrateur et homme de lettres....
1,http://data.bnf.fr/ark:/12148/cb13484444m#about,http://viaf.org/viaf/9999219,Gaston de Pawlowski,,1874,Docteur en droit. - Critique littéraire et thé...
2,http://data.bnf.fr/ark:/12148/cb134841632#about,http://viaf.org/viaf/9999131,Jean-Michel Berton,,1794,"Écrivain politique, avocat à la Cour de cassat..."
3,http://data.bnf.fr/ark:/12148/cb13379520q#about,http://viaf.org/viaf/9995247,Emmanuel Mathieu,,1852,"Docteur en droit (Paris, 1873)"
4,http://data.bnf.fr/ark:/12148/cb13338312g#about,http://viaf.org/viaf/9994322,Josiah Henry Benton,,1843,Juriste. - Bibliophile
5,http://data.bnf.fr/ark:/12148/cb13322315v#about,http://viaf.org/viaf/9991357,Paul Pic,,1862,Juriste. - Professeur de droit à la Faculté de...
6,http://data.bnf.fr/ark:/12148/cb13193319k#about,http://viaf.org/viaf/9989230,Gaston Ravisse,,1877,Avocat. - Spécialiste du monde de l'entreprise...
7,http://data.bnf.fr/ark:/12148/cb15042710d#about,http://viaf.org/viaf/99857689,Cândido Jucá Filho,,1900,Avocat
8,http://data.bnf.fr/ark:/12148/cb13169620f#about,http://viaf.org/viaf/9985289,Joseph de Trémaudan,,1846,Juge à Paimboeuf. - Historien local
9,http://data.bnf.fr/ark:/12148/cb13075767f#about,http://viaf.org/viaf/9982622,Achille Villey-Desmeserets,,1878,Avocat. - Préfet


In [29]:
df_bnf.drop_duplicates(subset ="uri_bnf", keep = 'first', inplace=True)
print(len(df_bnf))

9147


In [30]:
df_bnf.to_sql('df_bnf', con=engine)

ValueError: Table 'df_bnf' already exists.

In [129]:
df_dbp = pd.DataFrame([p for p in result_dbpedia if len(p[1]) > 0], columns=['uri_dbp', 'viaf', 'name_dbp', 'year_dbp'])
print(len(df_dbp))
df_dbp.fillna('')
df_dbp[-10:]

1747


,uri_dbp,viaf,name_dbp,year_dbp
1737,http://dbpedia.org/resource/Paul_A._Baran,http://viaf.org/viaf/100304945,Paul Alexander Baran,1909
1738,http://dbpedia.org/resource/Menachem_Elon,http://viaf.org/viaf/100286138,Menachem Elon,1923
1739,http://dbpedia.org/resource/József_Antall,http://viaf.org/viaf/100281452,József Antall,1932
1740,http://dbpedia.org/resource/Luiz_Carlos_Bresse...,http://viaf.org/viaf/100278275,Luiz Carlos Bresser-Pereira,1934
1741,http://dbpedia.org/resource/Michael_Kaser,http://viaf.org/viaf/100274542,Michael Kaser,1926
1742,http://dbpedia.org/resource/Merton_Miller,http://viaf.org/viaf/100258394,Merton Miller,1923
1743,http://dbpedia.org/resource/Peter_J._Hammond_(...,http://viaf.org/viaf/100246974,Peter Hammond,1945
1744,http://dbpedia.org/resource/José_Hermano_Saraiva,http://viaf.org/viaf/100238197,José Hermano Saraiva,1919
1745,http://dbpedia.org/resource/Alf_Ross,http://viaf.org/viaf/100233686,Alf Ross,1899
1746,http://dbpedia.org/resource/Max_Weber,http://viaf.org/viaf/100180950,,1864


In [32]:
df_dbp.drop_duplicates(subset ="uri_dbp", keep = 'last', inplace=True)
print(len(df_dbp))

1650


In [33]:
df_dbp.to_sql('df_dbp', con=engine)

ValueError: Table 'df_dbp' already exists.

In [130]:
df_wk= pd.DataFrame([p for p in result_wikidata if len(p[1]) > 0], columns=['uri_wk', 'viaf', 'name_wk', 'year_wk'])
print(len(df_wk))
df_wk.fillna('')
df_wk.head()

21975


,uri_wk,viaf,name_wk,year_wk
0,http://www.wikidata.org/entity/Q65561,http://viaf.org/viaf/232142151,Hans Apel,1932
1,http://www.wikidata.org/entity/Q65589,http://viaf.org/viaf/41917922,Alexander Rüstow,1885
2,http://www.wikidata.org/entity/Q91640,http://viaf.org/viaf/59144443,Werner Hildenbrand,1936
3,http://www.wikidata.org/entity/Q91836,http://viaf.org/viaf/30299539,Max Hirsch,1832
4,http://www.wikidata.org/entity/Q91603,http://viaf.org/viaf/40168611,Joachim Friedrich,1953


In [131]:
df_wk.drop_duplicates(subset ='uri_wk', keep = 'first', inplace=True)
print(len(df_wk))

21623


In [132]:
df_wk.to_sql('df_wk', con=engine)

ValueError: Table 'df_wk' already exists.

## Merge between different databases wich have a VIAF URI in common

### Between Wikidata and DBpedia

In [142]:
merged_df_1 = pd.merge( df_wk, df_dbp , on='viaf', how='inner', sort='viaf')
merged_df_1[:10]

,uri_wk,viaf,name_wk,year_wk,uri_dbp,name_dbp,year_dbp
0,http://www.wikidata.org/entity/Q9387,http://viaf.org/viaf/100180950,Max Weber,1864,http://dbpedia.org/resource/Max_Weber,,1864
1,http://www.wikidata.org/entity/Q15999850,http://viaf.org/viaf/100246974,Peter J. Hammond,1945,http://dbpedia.org/resource/Peter_J._Hammond_(...,Peter Hammond,1945
2,http://www.wikidata.org/entity/Q157255,http://viaf.org/viaf/100258394,Merton Miller,1923,http://dbpedia.org/resource/Merton_Miller,Merton Miller,1923
3,http://www.wikidata.org/entity/Q15990097,http://viaf.org/viaf/100274542,Michael Kaser,1926,http://dbpedia.org/resource/Michael_Kaser,Michael Kaser,1926
4,http://www.wikidata.org/entity/Q9025488,http://viaf.org/viaf/100278275,Luiz Carlos Bresser Pereira,1934,http://dbpedia.org/resource/Luiz_Carlos_Bresse...,Luiz Carlos Bresser-Pereira,1934
5,http://www.wikidata.org/entity/Q968139,http://viaf.org/viaf/100304945,Paul A. Baran,1909,http://dbpedia.org/resource/Paul_A._Baran,Paul Alexander Baran,1909
6,http://www.wikidata.org/entity/Q6794860,http://viaf.org/viaf/100595201,Max Hirsch,1852,http://dbpedia.org/resource/Max_Hirsch_(econom...,Max Hirsch,1852
7,http://www.wikidata.org/entity/Q2608419,http://viaf.org/viaf/10127341,Thomas Woods,1972,http://dbpedia.org/resource/Thomas_Woods,Thomas Woods,1972
8,http://www.wikidata.org/entity/Q78604,http://viaf.org/viaf/101860744,Fritz Machlup,1902,http://dbpedia.org/resource/Fritz_Machlup,Fritz Machlup,1902
9,http://www.wikidata.org/entity/Q11763850,http://viaf.org/viaf/101991628,Ludwik Landau,1902,http://dbpedia.org/resource/Ludwik_Maurycy_Landau,Ludwik Maurycy Landau,1902


In [143]:
print("the number of merged data from DBpedia and Wikidata is ",len(merged_df_1), "rows.")
print("")
print("The proportion of the number of merged data from DBpedia with Wikidata is ",((len(merged_df_1))/(len(df_dbp))*100),"%")
print("")
print("proportion of the number of merged data from Wikidata with DBpedia is ",((len(merged_df_1))/(len(df_wk))*100),"%")

the number of merged data from DBpedia and Wikidata is  932 rows.

The proportion of the number of merged data from DBpedia with Wikidata is  53.348597595878644 %

proportion of the number of merged data from Wikidata with DBpedia is  4.310225223142025 %


### Between Wikidata and BnF Data

In [144]:
merged_df_2 = pd.merge( df_wk, df_bnf , on='viaf', how='inner', sort='viaf')
print(len(merged_df_2))
merged_df_2[:10]

112


,uri_wk,viaf,name_wk,year_wk,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://www.wikidata.org/entity/Q540253,http://viaf.org/viaf/100277874,Clemens Maria Franz von Bönninghausen,1785,http://data.bnf.fr/ark:/12148/cb165146162#about,Clemens Maria Franz von Bönninghausen,,1785,"Avocat, agriculteur, botaniste, homéopathe. - ..."
1,http://www.wikidata.org/entity/Q11724800,http://viaf.org/viaf/101647977,Jerzy Karol Kurnatowski,1874,http://data.bnf.fr/ark:/12148/cb10528392p#about,Jerzy Kurnatowski,,1874,"Publiciste, juriste et économiste"
2,http://www.wikidata.org/entity/Q11738367,http://viaf.org/viaf/101863288,Kazimierz Studentowicz,1903,http://data.bnf.fr/ark:/12148/cb11261387v#about,Kazimierz Studentowicz,,1903,"Juriste, homme politique"
3,http://www.wikidata.org/entity/Q104820757,http://viaf.org/viaf/107036313,Félix Garcin,1879,http://data.bnf.fr/ark:/12148/cb13073916m#about,Félix Garcin,,1879,"Journaliste, directeur de ""Nouvelliste"", Lyon...."
4,http://www.wikidata.org/entity/Q1680590,http://viaf.org/viaf/107145857823423020439,Josef Redlich,1869,http://data.bnf.fr/ark:/12148/cb12818732h#about,Josef Redlich,,1869,Professeur de droit public et d'administration...
5,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,..."
6,http://www.wikidata.org/entity/Q88911,http://viaf.org/viaf/108482851,Otto Nathan,1893,http://data.bnf.fr/ark:/12148/cb12874301d#about,Otto Nathan,,1893,Économiste. - Avocat. - Exécuteur testamentair...
7,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...
8,http://www.wikidata.org/entity/Q87110924,http://viaf.org/viaf/110494713,Henri Denis,1913,http://data.bnf.fr/ark:/12148/cb12103181f#about,Henri Denis,,1913,"Docteur en droit (Paris, 1938). - Professeur d..."
9,http://www.wikidata.org/entity/Q1345621,http://viaf.org/viaf/111314151,Marco Biagi,1950,http://data.bnf.fr/ark:/12148/cb150888041#about,Marco Biagi,,1950,Juriste


In [145]:
print("The number of merged data from BnF Data and Wikidata is ",len(merged_df_2), "rows.")

print("")

print("The proportion of the number of merged data from BnF Data with Wikidata is ",((len(merged_df_2))/(len(df_bnf))*100),"%")

print("")

print("The proportion of the number of merged data from Wikidata with Wikidata is ",((len(merged_df_2))/(len(df_wk))*100),"%")

The number of merged data from BnF Data and Wikidata is  112 rows.

The proportion of the number of merged data from BnF Data with Wikidata is  1.224445173280857 %

The proportion of the number of merged data from Wikidata with Wikidata is  0.5179669796050502 %


### Between DBpedia and BnF Data

In [146]:
merged_df_3 = pd.merge( df_bnf, df_dbp , on='viaf', how='inner', sort='viaf')
print(len(merged_df_3))
merged_df_3[:10]

93


,uri_bnf,viaf,name_bnf,sName,year_bnf,bio_bnf,uri_dbp,name_dbp,year_dbp
0,http://data.bnf.fr/ark:/12148/cb122145877#about,http://viaf.org/viaf/100966624,John Humphrey,,1905,Juriste. - A été professeur de droit internati...,http://dbpedia.org/resource/John_Peters_Humphrey,John Peters Humphrey,1905
1,http://data.bnf.fr/ark:/12148/cb12327654n#about,http://viaf.org/viaf/107536763,Louis Renault,,1843,Juriste. - Professeur de droit international à...,http://dbpedia.org/resource/Louis_Renault_(jur...,Louis Renault,1843
2,http://data.bnf.fr/ark:/12148/cb122775427#about,http://viaf.org/viaf/108173876,Ronald Myles Dworkin,,1931,Juriste. - Professeur de jurisprudence à la Ya...,http://dbpedia.org/resource/Ronald_Dworkin,,1931
3,http://data.bnf.fr/ark:/12148/cb11927239j#about,http://viaf.org/viaf/108188941,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,...",http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922
4,http://data.bnf.fr/ark:/12148/cb120906270#about,http://viaf.org/viaf/108565309,Paul Abraham Freund,,1908,"Professeur de droit, ""Harvard Law School""",http://dbpedia.org/resource/Paul_A._Freund,Paul Abraham Freund,1908
5,http://data.bnf.fr/ark:/12148/cb119084288#about,http://viaf.org/viaf/108587991,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938
6,http://data.bnf.fr/ark:/12148/cb128832222#about,http://viaf.org/viaf/108624624,Muḥammad Ẓafr Allāh H̱ān,,1893,"Juriste, diplomate et homme politique",http://dbpedia.org/resource/Muhammad_Zafarulla...,CH Muhammad Zafarullah Khan,1893
7,http://data.bnf.fr/ark:/12148/cb12299375j#about,http://viaf.org/viaf/108794549,Karl Engisch,,1899,Juriste. - Spécialiste de philosophie du droit...,http://dbpedia.org/resource/Karl_Engisch,Karl Engisch,1899
8,http://data.bnf.fr/ark:/12148/cb118935370#about,http://viaf.org/viaf/111389197,Georges Bousquet,,1846,Avocat au Barreau de Paris (en 1866). - Engagé...,http://dbpedia.org/resource/Georges_Hilaire_Bo...,Georges Hilaire Bousquet,1845
9,http://data.bnf.fr/ark:/12148/cb12328362p#about,http://viaf.org/viaf/11396531,John Paul Stevens,,1920,Juriste américain,http://dbpedia.org/resource/John_Paul_Stevens,John Paul Stevens,1920


In [147]:
print("The number of merged data from BnF Data and Wikidata is ",len(merged_df_3), "rows.")

print("")

print("The proportion of the number of merged data from BnF Data with DBpedia is ",((len(merged_df_3))/(len(df_dbp))*100),"%")

print("")

print("The proportion of the number of merged data from DBpedia with BnF Data is ",((len(merged_df_3))/(len(df_bnf))*100),"%")

The number of merged data from BnF Data and Wikidata is  93 rows.

The proportion of the number of merged data from BnF Data with DBpedia is  5.323411562678878 %

The proportion of the number of merged data from DBpedia with BnF Data is  1.0167267956707118 %


### Between Wikidata, BnF Data and DBpedia

In [148]:
merged_df = pd.merge( merged_df_1, df_bnf , on='viaf', how='inner', sort='viaf')
merged_df[:10]

,uri_wk,viaf,name_wk,year_wk,uri_dbp,name_dbp,year_dbp,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,..."
1,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...
2,http://www.wikidata.org/entity/Q3085838,http://viaf.org/viaf/32062931,François Simiand,1873,http://dbpedia.org/resource/François_Simiand,François Simiand,1873,http://data.bnf.fr/ark:/12148/cb12301152q#about,François Simiand,,1873,Philosophe. - Agrégé de philosophie. - Docteur...
3,http://www.wikidata.org/entity/Q61956,http://viaf.org/viaf/44308789,Lorenz von Stein,1815,http://dbpedia.org/resource/Lorenz_von_Stein,Lorenz von Stein,1815,http://data.bnf.fr/ark:/12148/cb12001622n#about,Lorenz von Stein,,1815,"Juriste et économiste. - Professeur à Kiel, Al..."
4,http://www.wikidata.org/entity/Q231690,http://viaf.org/viaf/44331988,B. R. Ambedkar,1891,http://dbpedia.org/resource/B._R._Ambedkar,Bhimrao Ramji Ambedkar,1891,http://data.bnf.fr/ark:/12148/cb12126992f#about,Bhimrao Ramji Ambedkar,,1891,Homme politique d'origine harijan mahar. - Étu...
5,http://www.wikidata.org/entity/Q215961,http://viaf.org/viaf/50021033,Franz Hermann Schulze-Delitzsch,1808,http://dbpedia.org/resource/Franz_Hermann_Schu...,Hermann Schulze-Delitzsch,1808,http://data.bnf.fr/ark:/12148/cb12088660j#about,Hermann Schulze-Delitzsch,,1808,"Juriste, homme politique et économiste alleman..."
6,http://www.wikidata.org/entity/Q4893263,http://viaf.org/viaf/69263532,Joan Sardà i Dexeus,1910,http://dbpedia.org/resource/Joan_Sardà_i_Dexeus,Joan Sardà i Dexeus,1910,http://data.bnf.fr/ark:/12148/cb158098327#about,Juan Sardá Dexeus,,1910,Docteur en droit. - Économiste
7,http://www.wikidata.org/entity/Q7836141,http://viaf.org/viaf/73921034,Travers Twiss,1809,http://dbpedia.org/resource/Travers_Twiss,Travers Twiss,1809,http://data.bnf.fr/ark:/12148/cb12314495r#about,Travers Twiss,,1809,Juriste. - Spécialiste de droit international


In [149]:
print("The number of merged data from DBpedia, Wikidata and BnF Data is",len(merged_df),"rows.")
print("")
print("The proportion of the number of merged data from DBpedia, Wikidata and BnF Data is ",(len(merged_df))/(len(df_bnf))*100,"%" )

The number of merged data from DBpedia, Wikidata and BnF Data is 8 rows.

The proportion of the number of merged data from DBpedia, Wikidata and BnF Data is  0.08746036952006123 %


### Append DBpedia, BnF Data and Wikidata to the merged Dataframe who don't have viaf.org values in common

In [150]:
# To achieve this, I am served on these pages: 
# https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html (en)
# https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html (en)
# http://www.python-simple.com/python-pandas/concatenations-joins-dataframe.php (fr)

result = merged_df.append([merged_df_1, merged_df_2, merged_df_3, df_bnf, df_wk, df_dbp], sort=False)
print(len(result))
result[:10]

33662


,uri_wk,viaf,name_wk,year_wk,uri_dbp,name_dbp,year_dbp,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,..."
1,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...
2,http://www.wikidata.org/entity/Q3085838,http://viaf.org/viaf/32062931,François Simiand,1873,http://dbpedia.org/resource/François_Simiand,François Simiand,1873,http://data.bnf.fr/ark:/12148/cb12301152q#about,François Simiand,,1873,Philosophe. - Agrégé de philosophie. - Docteur...
3,http://www.wikidata.org/entity/Q61956,http://viaf.org/viaf/44308789,Lorenz von Stein,1815,http://dbpedia.org/resource/Lorenz_von_Stein,Lorenz von Stein,1815,http://data.bnf.fr/ark:/12148/cb12001622n#about,Lorenz von Stein,,1815,"Juriste et économiste. - Professeur à Kiel, Al..."
4,http://www.wikidata.org/entity/Q231690,http://viaf.org/viaf/44331988,B. R. Ambedkar,1891,http://dbpedia.org/resource/B._R._Ambedkar,Bhimrao Ramji Ambedkar,1891,http://data.bnf.fr/ark:/12148/cb12126992f#about,Bhimrao Ramji Ambedkar,,1891,Homme politique d'origine harijan mahar. - Étu...
5,http://www.wikidata.org/entity/Q215961,http://viaf.org/viaf/50021033,Franz Hermann Schulze-Delitzsch,1808,http://dbpedia.org/resource/Franz_Hermann_Schu...,Hermann Schulze-Delitzsch,1808,http://data.bnf.fr/ark:/12148/cb12088660j#about,Hermann Schulze-Delitzsch,,1808,"Juriste, homme politique et économiste alleman..."
6,http://www.wikidata.org/entity/Q4893263,http://viaf.org/viaf/69263532,Joan Sardà i Dexeus,1910,http://dbpedia.org/resource/Joan_Sardà_i_Dexeus,Joan Sardà i Dexeus,1910,http://data.bnf.fr/ark:/12148/cb158098327#about,Juan Sardá Dexeus,,1910,Docteur en droit. - Économiste
7,http://www.wikidata.org/entity/Q7836141,http://viaf.org/viaf/73921034,Travers Twiss,1809,http://dbpedia.org/resource/Travers_Twiss,Travers Twiss,1809,http://data.bnf.fr/ark:/12148/cb12314495r#about,Travers Twiss,,1809,Juriste. - Spécialiste de droit international
0,http://www.wikidata.org/entity/Q9387,http://viaf.org/viaf/100180950,Max Weber,1864,http://dbpedia.org/resource/Max_Weber,,1864,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q15999850,http://viaf.org/viaf/100246974,Peter J. Hammond,1945,http://dbpedia.org/resource/Peter_J._Hammond_(...,Peter Hammond,1945,NaN,NaN,NaN,NaN,NaN


In [152]:
# Replace each null value with the value from the other database.
# Create a column with the fresh values.
# I used this page: https://kanoki.org/2019/08/17/pandas-coalesce-replace-value-from-another-column/ 
# particuliarly, the "Using Numpy" section
result_test=result

# Replace null value name

## BnF Data
result_test['name'] = np.where(result['name_bnf'].isnull(),result['name_dbp'],result['name_bnf'])
result_test['name'] = np.where(result['name_bnf'].isnull(),result['name_wk'],result['name_bnf'])
## DBpedia
result_test['name'] = np.where(result['name_dbp'].isnull(),result['name_bnf'],result['name_dbp'])
result_test['name'] = np.where(result['name_dbp'].isnull(),result['name_wk'],result['name_dbp'])
## Wikidata
result_test['name'] = np.where(result['name_wk'].isnull(),result['name_bnf'],result['name_wk'])
result_test['name'] = np.where(result['name_wk'].isnull(),result['name_dbp'],result['name_wk'])

# Replace null value year

## BnF Data
result_test['year'] = np.where(result['year_bnf'].isnull(),result['year_dbp'],result['year_bnf'])
result_test['year'] = np.where(result['year_bnf'].isnull(),result['year_wk'],result['year_bnf'])
## DBpedia
result_test['year'] = np.where(result['year_dbp'].isnull(),result['year_bnf'],result['year_dbp'])
result_test['year'] = np.where(result['year_dbp'].isnull(),result['year_wk'],result['year_dbp'])
## Wikidata
result_test['year'] = np.where(result['year_wk'].isnull(),result['year_bnf'],result['year_wk'])
result_test['year'] = np.where(result['year_wk'].isnull(),result['year_dbp'],result['year_wk'])

result_test=result_test.loc[:,['viaf','name','year','uri_dbp','uri_bnf', 'uri_wk']]
result_test[:5]

,viaf,name,year,uri_dbp,uri_bnf,uri_wk
0,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://dbpedia.org/resource/Gordon_Tullock,http://data.bnf.fr/ark:/12148/cb11927239j#about,http://www.wikidata.org/entity/Q518859
1,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://dbpedia.org/resource/Alexis_Jacquemin,http://data.bnf.fr/ark:/12148/cb119084288#about,http://www.wikidata.org/entity/Q652154
2,http://viaf.org/viaf/32062931,François Simiand,1873,http://dbpedia.org/resource/François_Simiand,http://data.bnf.fr/ark:/12148/cb12301152q#about,http://www.wikidata.org/entity/Q3085838
3,http://viaf.org/viaf/44308789,Lorenz von Stein,1815,http://dbpedia.org/resource/Lorenz_von_Stein,http://data.bnf.fr/ark:/12148/cb12001622n#about,http://www.wikidata.org/entity/Q61956
4,http://viaf.org/viaf/44331988,B. R. Ambedkar,1891,http://dbpedia.org/resource/B._R._Ambedkar,http://data.bnf.fr/ark:/12148/cb12126992f#about,http://www.wikidata.org/entity/Q231690


In [153]:
# Extract only rows without VIAF uri from DBpeida
df_dbp_test = pd.DataFrame(result_dbpedia, columns=['uri_dbp', 'viaf', 'name_dbp', 'year_dbp'])

df_dbp_test_mask=df_dbp_test['viaf']==''
filtered_df_dbp_test = df_dbp_test[df_dbp_test_mask]
print(len(filtered_df_dbp_test))
filtered_df_dbp_test[:10]

6798


,uri_dbp,viaf,name_dbp,year_dbp
1747,http://dbpedia.org/resource/Luc-Normand_Tellier,,Luc-Normand Tellier,1944
1748,http://dbpedia.org/resource/Madhu_Verma,,Madhu Verma,1961
1749,http://dbpedia.org/resource/Magda_Kandil,,Magda ElSayed Kandil,1958
1750,http://dbpedia.org/resource/Magnus_Johannesson,,Magnus Johannesson,1964
1751,http://dbpedia.org/resource/Mahendra_P._Lama,,Mahendra P. Lama,1961
1752,http://dbpedia.org/resource/Mainul_Islam,,Mainul Islam,1950
1753,http://dbpedia.org/resource/Urs_Meisterhans,,Urs Meisterhans,1960
1754,http://dbpedia.org/resource/Rosalind_Blauer,,Rosalind Blauer,1943
1755,http://dbpedia.org/resource/Makoto_Yano,,Makoto Yano,1952
1756,http://dbpedia.org/resource/Krzysztof_Zamasz,,Krzysztof Zamasz,1974


In [154]:
# drop duplicates in DBpedia dataframe
filtered_df_dbp_test.drop_duplicates(subset ="uri_dbp", keep = False, inplace=True)
print(len(filtered_df_dbp_test))

6320


/home/sylvain/anaconda3/envs/Github/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [155]:
# Extract only rows without VIAF uri from BnF Data
df_bnf_test = pd.DataFrame(result_bnf, columns=['uri_bnf', 'viaf', 'name_bnf', 'sName','year_bnf','bio_bnf'])

df_bnf_test_mask=df_bnf_test['viaf']==''
filtered_df_bnf_test = df_bnf_test[df_bnf_test_mask]
len(filtered_df_bnf_test)

2054

In [156]:
# drop duplicates in BnF Data dataframe

filtered_df_bnf_test.drop_duplicates(subset ="uri_bnf", keep = 'first', inplace=True)
print(len(filtered_df_bnf_test))

2054


/home/sylvain/anaconda3/envs/Github/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
##### Extract only rows without VIAF uri from BnF Data
df_wk_test = pd.DataFrame(result_wikidata, columns=['uri_wk', 'viaf', 'name_wk','year_wk'])

df_wk_test_mask=df_wk_test['viaf']==''
filtered_df_wk_test = df_wk_test[df_wk_test_mask]
len(filtered_df_wk_test)

7437

In [159]:
# drop duplicates in Wikidata dataframe

filtered_df_wk_test.drop_duplicates(subset ="uri_wk", keep = 'first', inplace=True)
print(len(filtered_df_wk_test))

7418


/home/sylvain/anaconda3/envs/Github/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [160]:
# merge the three dateframes 
filtered_dbp_bnf_test= filtered_df_dbp_test.append(filtered_df_bnf_test, sort=True)
len(filtered_dbp_bnf_test)
filtered_dbp_bnf_wk_test=filtered_dbp_bnf_test.append(filtered_df_wk_test, sort=True)
print(len(filtered_dbp_bnf_wk_test))
filtered_dbp_bnf_wk_test[-1500:]

15792


,bio_bnf,name_bnf,name_dbp,name_wk,sName,uri_bnf,uri_dbp,uri_wk,viaf,year_bnf,year_dbp,year_wk
27124,NaN,NaN,NaN,Thomas G. G. Senghore,NaN,NaN,NaN,http://www.wikidata.org/entity/Q96589687,,NaN,NaN,1936
27125,NaN,NaN,NaN,Ahmed Kariye,NaN,NaN,NaN,http://www.wikidata.org/entity/Q96627421,,NaN,NaN,1968
27126,NaN,NaN,NaN,Fábio Rodrigues de Moura,NaN,NaN,NaN,http://www.wikidata.org/entity/Q96628921,,NaN,NaN,1986
27127,NaN,NaN,NaN,Carlos Decotelli,NaN,NaN,NaN,http://www.wikidata.org/entity/Q96643922,,NaN,NaN,1949
27128,NaN,NaN,NaN,Ap Reinders,NaN,NaN,NaN,http://www.wikidata.org/entity/Q79414114,,NaN,NaN,1973
27137,NaN,NaN,NaN,Nikolay Vladimirovich Omelyanovich,NaN,NaN,NaN,http://www.wikidata.org/entity/Q79357795,,NaN,NaN,1937
27146,NaN,NaN,NaN,Liu Hechen,NaN,NaN,NaN,http://www.wikidata.org/entity/Q78116601,,NaN,NaN,1905
27147,NaN,NaN,NaN,Song Zexing,NaN,NaN,NaN,http://www.wikidata.org/entity/Q78116650,,NaN,NaN,1917
27148,NaN,NaN,NaN,Marco Brogna,NaN,NaN,NaN,http://www.wikidata.org/entity/Q78035469,,NaN,NaN,1968
27149,NaN,NaN,NaN,Wan Dianwu,NaN,NaN,NaN,http://www.wikidata.org/entity/Q78116840,,NaN,NaN,1921


In [172]:
# Replace each null value with the value from the other database.
# Create a column with the fresh values.
# I used this page: https://kanoki.org/2019/08/17/pandas-coalesce-replace-value-from-another-column/ 
# particuliarly, the "Using Numpy" section

result_f_test=filtered_dbp_bnf_wk_test

# Replace null value name

## BnF Data
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_bnf'].isnull(),filtered_dbp_bnf_wk_test['name_dbp'],filtered_dbp_bnf_wk_test['name_bnf'])
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_bnf'].isnull(),filtered_dbp_bnf_wk_test['name_wk'],filtered_dbp_bnf_wk_test['name_bnf'])
## DBpedia
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_dbp'].isnull(),filtered_dbp_bnf_wk_test['name_bnf'],filtered_dbp_bnf_wk_test['name_dbp'])
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_dbp'].isnull(),filtered_dbp_bnf_wk_test['name_wk'],filtered_dbp_bnf_wk_test['name_dbp'])
## Wikidata
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_wk'].isnull(),filtered_dbp_bnf_wk_test['name_bnf'],filtered_dbp_bnf_wk_test['name_wk'])
result_f_test['name'] = np.where(filtered_dbp_bnf_wk_test['name_wk'].isnull(),filtered_dbp_bnf_wk_test['name_dbp'],filtered_dbp_bnf_wk_test['name_wk'])

# Replace null value year

## BnF Data
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_bnf'].isnull(),filtered_dbp_bnf_wk_test['year_dbp'],filtered_dbp_bnf_wk_test['year_bnf'])
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_bnf'].isnull(),filtered_dbp_bnf_wk_test['year_wk'],filtered_dbp_bnf_wk_test['year_bnf'])
## DBpedia
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_dbp'].isnull(),filtered_dbp_bnf_wk_test['year_bnf'],filtered_dbp_bnf_wk_test['year_dbp'])
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_dbp'].isnull(),filtered_dbp_bnf_wk_test['year_wk'],filtered_dbp_bnf_wk_test['year_dbp'])
## Wikidata
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_wk'].isnull(),filtered_dbp_bnf_wk_test['year_bnf'],filtered_dbp_bnf_wk_test['year_wk'])
result_f_test['year'] = np.where(filtered_dbp_bnf_wk_test['year_wk'].isnull(),filtered_dbp_bnf_wk_test['year_dbp'],filtered_dbp_bnf_wk_test['year_wk'])


result_f_test=result_f_test.loc[:,['viaf','name','year','uri_dbp','uri_bnf', 'uri_wk']]
sort_rft=result_f_test.sort_values(by='name', ascending=False)
sort_rft[:10]

,viaf,name,year,uri_dbp,uri_bnf,uri_wk
17882,,Александр Валерьевич Дубилет,1962,NaN,NaN,http://www.wikidata.org/entity/Q20066943
21379,,Απέργης Νικόλαος,1962,NaN,NaN,http://www.wikidata.org/entity/Q38597547
25283,,ʻAbd al-Ḥusayn Waddāy al-ʻAṭīyah,1929,NaN,NaN,http://www.wikidata.org/entity/Q66428907
5741,,Əvəz Ələkbərov,1952,NaN,NaN,http://www.wikidata.org/entity/Q1099741
16846,,Željko Topić,1959,NaN,NaN,http://www.wikidata.org/entity/Q17402923
4114,,Štefan Tiso,1897,http://dbpedia.org/resource/Štefan_Tiso,NaN,NaN
4113,,Štefan Osuský,1889,http://dbpedia.org/resource/Štefan_Osuský,NaN,NaN
23017,,Štefan Bukovec,1929,NaN,NaN,http://www.wikidata.org/entity/Q59851859
15916,,Şəfa Əliyev,1959,NaN,NaN,http://www.wikidata.org/entity/Q12849692
21504,,Şükrü Kızılot,1958,NaN,NaN,http://www.wikidata.org/entity/Q38170668


In [173]:
result_final = sort_rft.append([result_test], sort=False)
print(len(result_final))
result_final[-10:]

49454


,viaf,name,year,uri_dbp,uri_bnf,uri_wk
1737,http://viaf.org/viaf/100304945,Paul Alexander Baran,1909,http://dbpedia.org/resource/Paul_A._Baran,NaN,NaN
1738,http://viaf.org/viaf/100286138,Menachem Elon,1923,http://dbpedia.org/resource/Menachem_Elon,NaN,NaN
1739,http://viaf.org/viaf/100281452,József Antall,1932,http://dbpedia.org/resource/József_Antall,NaN,NaN
1740,http://viaf.org/viaf/100278275,Luiz Carlos Bresser-Pereira,1934,http://dbpedia.org/resource/Luiz_Carlos_Bresse...,NaN,NaN
1741,http://viaf.org/viaf/100274542,Michael Kaser,1926,http://dbpedia.org/resource/Michael_Kaser,NaN,NaN
1742,http://viaf.org/viaf/100258394,Merton Miller,1923,http://dbpedia.org/resource/Merton_Miller,NaN,NaN
1743,http://viaf.org/viaf/100246974,Peter Hammond,1945,http://dbpedia.org/resource/Peter_J._Hammond_(...,NaN,NaN
1744,http://viaf.org/viaf/100238197,José Hermano Saraiva,1919,http://dbpedia.org/resource/José_Hermano_Saraiva,NaN,NaN
1745,http://viaf.org/viaf/100233686,Alf Ross,1899,http://dbpedia.org/resource/Alf_Ross,NaN,NaN
1746,http://viaf.org/viaf/100180950,,1864,http://dbpedia.org/resource/Max_Weber,NaN,NaN


In [174]:
# Drop duplicates while preserving NaN values
# cf. https://stackoverflow.com/questions/23512339/drop-duplicates-while-preserving-nans-in-pandas

## DBpedia
result_final=result_final[result_final['uri_dbp'].isnull() | ~result_final.duplicated(subset='uri_dbp',keep='first')]
## Wikidata
result_final=result_final[result_final['uri_wk'].isnull() | ~result_final.duplicated(subset='uri_wk',keep='first')]
## BnF_Data
result_final=result_final[result_final['uri_bnf'].isnull() | ~result_final.duplicated(subset='uri_bnf',keep='last')]

result_final=result_final.sort_values(by='name', ascending=False)
print(len(result_final))
result_final[-10:]


46903


,viaf,name,year,uri_dbp,uri_bnf,uri_wk
9137,http://viaf.org/viaf/100167417,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb12328758j#about,NaN
9138,http://viaf.org/viaf/100164495,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb10462070r#about,NaN
9139,http://viaf.org/viaf/100160882,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb12348029f#about,NaN
9140,http://viaf.org/viaf/100149617040103751310,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb13004088s#about,NaN
9141,http://viaf.org/viaf/100144782962367904089,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb11848362t#about,NaN
9142,http://viaf.org/viaf/100061313,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb13010553r#about,NaN
9143,http://viaf.org/viaf/10004221,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb135169833#about,NaN
9144,http://viaf.org/viaf/100040132,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb11172762w#about,NaN
9145,http://viaf.org/viaf/100008683,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb16607520q#about,NaN
9146,http://viaf.org/viaf/10000386,NaN,NaN,NaN,http://data.bnf.fr/ark:/12148/cb134881720#about,NaN


In [171]:
# Test to find similarities between the names BnF Data, DBpedia and Wikidata with collocation 


from nltk.corpus import stopwords
stopset = stopwords.words('english')
stopset = stopwords.words('french')

result_final['tokenized_sents'] = result_f_test.apply(lambda row: nltk.word_tokenize(row['name']), axis=1)

bcf = TrigramCollocationFinder.from_documents(result_final['tokenized_sents'])
filter_stops = lambda w: w in stopset
bcf.apply_word_filter(filter_stops)
f= bcf.nbest(TrigramAssocMeasures, 100)
print(f)

TypeError: ('expected string or bytes-like object', 'occurred at index 9147')

In [42]:
# attempt to change the 'NaN' names from DBpedia to name from BnF and the same for the reverse 
# (cf.https://stackoverflow.com/questions/29177498/python-pandas-replace-nan-in-one-column-with-value-from-corresponding-row-of-sec)
import pandas as pd
import numpy as np

def fx(x):
    if np.isnan(x['date_dbp']):
        return x['year_bnf ']
    else:
        return x['date_dbp']
result_test=result.apply(lambda x : fx(x),axis=1)

TypeError: ("ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''", 'occurred at index 0')

In [43]:
# Another method for trying to do it 
result.name_dbp.fillna(result.name_bnf, inplace=True)
del result['name_bnf']
result.columns = 'uri_dbp viaf name_dbp date_dbp uri_bnf sName year_bnf bio_bnf'.split()


ValueError: Length mismatch: Expected axis has 10 elements, new values have 8 elements

In [42]:
df1 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005', 'id006', 'id007'],
                    'first_name': ['Rivi', 'Wynnie', 'Kristos', 'Madalyn', 'Tobe', 'Regan', 'Kristin'],
                    'last_name': ['Valti', 'McMurty', 'Ivanets', 'Max', 'Riddich', 'Huyghe', 'Illis'],
                    'email': ['rvalti0@example.com', 'wmcmurty1@example.com', 'kivanets2@example.com',
                              'mmax3@example.com', 'triddich4@example.com', 'rhuyghe@example.com', 'killis4@example.com']
                    })

In [43]:
df2 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005'],
                    'image_url': ['http://example.com/img/id001.png', 'http://example.com/img/id002.jpg',
                                  'http://example.com/img/id003.bmp', 'http://example.com/img/id004.jpg',
                                  'http://example.com/img/id005.png']
                    })

In [11]:
df3_merged = pd.merge(df1, df2)
df3_merged 

,user_id,first_name,last_name,email,image_url
0,id001,Rivi,Valti,rvalti0@example.com,http://example.com/img/id001.png
1,id002,Wynnie,McMurty,wmcmurty1@example.com,http://example.com/img/id002.jpg
2,id003,Kristos,Ivanets,kivanets2@example.com,http://example.com/img/id003.bmp
3,id004,Madalyn,Max,mmax3@example.com,http://example.com/img/id004.jpg
4,id005,Tobe,Riddich,triddich4@example.com,http://example.com/img/id005.png
